In [89]:
import pickle5 as pickle
import numpy as np

T = 200
p = 1
n_iterations = 200
prior = "svss"
sparsity = 0.40

In [90]:
with open(f'../simulations/results/recalculation/statistics_{T}_{p}_{n_iterations}_{prior}_{sparsity}.pkl', 'rb') as f:
        simulation_result = pickle.load(f)

In [91]:
# Select one of the results from the TVP for a comparison

# tvp_bar_result = np.array([1.409606, 1.439558, 1.443909, 1.460863, 1.459694, 1.461891, 1.460617, 1.463583]) # 100 0.20 M = 3
# tvp_bar_result = np.array([1.425317, 1.451723, 1.453873, 1.461871, 1.463032, 1.469662, 1.463004, 1.465282]) # 100 0.40 M = 3
# tvp_bar_result = np.array([1.744158, 1.751534, 1.762016, 1.755724, 1.759277, 1.776115, 1.780216, 1.790951]) # 200 0.20 M = 3
# tvp_bar_result = np.array([1.763981, 1.769919, 1.783354, 1.783171, 1.788644, 1.803967, 1.804127, 1.821872]) # 200 0.40 M = 3

# tvp_bar_result = np.array([2.203113, 2.266640, 2.307729, 2.323546, 2.341028, 2.362956, 2.385565, 2.401525]) # 100 0.20 M = 7
# tvp_bar_result = np.array([2.240987, 2.319214, 2.354877, 2.361660, 2.384902, 2.406189, 2.429912, 2.450251]) # 100 0.40 M = 7
# tvp_bar_result = np.array([3.041940, 3.091507, 3.119626, 3.138423, 3.151080, 3.172323, 3.196318, 3.218827]) # 200 0.20 M = 7
tvp_bar_result = np.array([3.185229, 3.270465, 3.310410, 3.340728, 3.399988, 3.452003, 3.507754, 3.621743]) # 200 0.40 M = 7

In [92]:
simulation_result_block = np.block(simulation_result[0])

In [93]:
M_3 = simulation_result_block[:n_iterations]
M_7 = simulation_result_block[n_iterations:]

In [94]:
result_block = M_7

threshold_percentage = 10e-2

threshold_high = int(round(n_iterations - threshold_percentage*n_iterations,0))

msfe_list = result_block[:,:8].astype(np.float64).mean(1)

cleaned_indices = msfe_list.argsort()[:threshold_high]
msfe_list_cleaned = msfe_list[cleaned_indices]

msfe_h_step = result_block[cleaned_indices,:8].astype(np.float64).mean(0)

msd = result_block[cleaned_indices,16].astype(np.float64).mean()

In [95]:
msd = round(msd,6)
msfe = round(np.mean(msfe_list_cleaned),3)

print(f"MSFE = {msfe} & MSD = {msd}")

MSFE = 4.612 & MSD = 0.029172


In [74]:
msfe_ratio = np.round(msfe_h_step/tvp_bar_result,3)

print(f"VI/TVP = {msfe_ratio}")

VI/TVP = [0.975 1.387 1.387 1.394 1.379 1.364 1.348 1.314]


In [88]:
msfe_h_step.mean()/tvp_bar_result.mean()

1.3208067870938531

### Random walk (RW)

In [37]:
def calculate_rw_msfe(y, T, train, total_h=8):
    
    train_index = train
    number_of_predictions = T - train_index
    msfe_tvp = np.zeros(total_h)
    total_h = np.arange(1,total_h+1)

    for h in total_h:
            y_true_h = y[:,train_index:]
            y_pred_h = y[:,(train_index - h):-h]

            msfe_tvp[h-1] = np.mean((y_pred_h - y_true_h) ** 2)
            
    return msfe_tvp

In [38]:

M = 7
train = T - 25
total_h = 8
n_iterations = 200
iterations_set = np.arange(1,n_iterations+1,1)
msfe_rw = np.zeros((n_iterations, total_h))

for run in iterations_set:

    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    msfe_rw[run-1] = calculate_rw_msfe(y, T, train, total_h)

In [39]:

threshold_percentage = 2.5e-2

threshold_high = int(round(n_iterations - threshold_percentage*n_iterations,0))

cleaned_indices = msfe_rw.mean(1).argsort()[:threshold_high]
msfe_rw_cleaned = msfe_rw[cleaned_indices]

h_step_rw = msfe_rw_cleaned.mean(0)

In [43]:
msfe_ratio = np.round(h_step_rw/tvp_bar_result,3)

print(f"RW/TVP = {msfe_ratio} \nMSFE: {round(h_step_rw.mean(),4)}")

RW/TVP = [1.763 2.372 2.762 3.03  3.26  3.442 3.589 3.733] 
MSFE: 9.4373


In [42]:
msfe_ratio = np.round(h_step_rw/msfe_h_step,3)

print(f"RW/VI = {msfe_ratio} \nMSFE: {round(h_step_rw.mean(),4)}")

RW/VI = [1.726 1.616 1.877 2.042 2.191 2.319 2.425 2.524] 
MSFE: 9.4373
